This script evaluates the performance of different graph-aware architectures in a node classification problem. Several datasets are employed paying special attention to the homophily ratio.

In [1]:
import time
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import dgl
import networkx as nx
import torch
import torch.nn as nn


import utils
from gsp_utils.baselines_archs import GCNN_2L
from gsp_utils.baselines_models import NodeClassModel, GF_NodeClassModel
from gsp_utils.data import normalize_gso
from src.arch import GFGCN, GFGCNLayer, GFGCN_noh_Layer, GFGCN_Spows  

# SEED = 0
SEED = 15
PATH = 'results/diff_filters/'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

torch.manual_seed(SEED)

cpu


In [2]:
def print_full_results(accs, ellapsed_time, datasets, exps):
    mean_accs = accs.mean(axis=2)
    med_accs = np.median(accs, axis=2)
    std_accs = accs.std(axis=2)
    mean_t = ellapsed_time.mean(axis=2)

    for i, dataset_name in enumerate(datasets):
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])

        print(f'{dataset_name} (homophily ratio: {edge_hom:.3f})')
        for j, exp in enumerate(exps):
            print(f'\t- {exp["leg"]}:\tmean: {mean_accs[j,i]:.3f} - std: {std_accs[j,i]:.4f} - med: {med_accs[j,i]:.3f} - time: {mean_t[j,i]:.2f} mins')
        
        print()

def summary_table(accs, datasets, exps, median=False):
    mean_accs = accs.mean(axis=2)
    cols_name = []
    for dataset_name in datasets:
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])
        cols_name.append(f'{dataset_name} ({edge_hom:.2f})')

    index_name = [exp['leg'] for exp in exps]

    return DataFrame(mean_accs, columns=cols_name, index=index_name)


In [3]:
DATASETS = ['TexasDataset',  'WisconsinDataset', 'CornellDataset', 'CiteseerGraphDataset', 'CoraGraphDataset']

# EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
#         {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
#         {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': .01, 'norm': True, 'leg': 'GCN-normA'},
#         {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': .01, 'norm': False, 'leg': 'GCN-A'},
#         {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'GCN-normA-1'},
#         {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'GCN-A-1'},
#         {'model': 'GFGCN', 'L': 4, 'K': 2, 'h0': .01, 'norm': False, 'leg': 'GCN-A-4-2'},
#         {'model': 'H-GFGCN', 'L': 2, 'K': 2, 'norm': True, 'leg': 'GCN-normH'},
#         {'model': 'H-GFGCN', 'L': 2, 'K': 2, 'norm': False, 'leg': 'GCN-H'},
#         {'model': 'noh-GFGCN', 'L': 2, 'K': 2, 'norm': True, 'leg': 'W-GCN-normA'},
#         {'model': 'noh-GFGCN', 'L': 2, 'K': 2, 'norm': False, 'leg': 'W-GCN-A'},]

## Best params

In [4]:
N_RUNS = 10
ACT = nn.LeakyReLU()  # nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss() #nn.NLLLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-A'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 3, 'h0': 1, 'norm': True, 'leg': 'A-GCN-normA'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 32,'L': 2, 'K': 3, 'h0': 1, 'norm': False, 'leg': 'A-GCN'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 50,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'A-GCN-v2'},

        {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 16,'L': 3, 'K': 2, 'norm': True, 'leg': 'H-GCN-normH'},

        {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 32,'L': 2, 'K': 3, 'norm': False, 'leg': 'H-GCN'},

        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .05, 'wd': .005, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 3, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .05, 'wd': .005, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 3, 'norm': False, 'leg': 'W-GCN'},
      ]


In [5]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, exp['hid_dim'], out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=exp['drop'], norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=exp['drop'], diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=exp['drop'], norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=exp['drop'], diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'])
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'],
                                        epochs_h=exp['e_h'], epochs_W=exp['e_W'])
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc1 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val1 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val1b = summary_table(accs_best_val2, DATASETS, EXPS)

TexasDataset
0: 

In [ ]:
table_acc_val1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.586486,0.539216,0.475676,0.5973,0.5589
Kipf-A,0.624324,0.529412,0.440541,0.6845,0.7845
A-GCN-normA,0.156757,0.037255,0.240541,0.7048,0.8062
A-GCN,0.816216,0.835294,0.737838,0.6043,0.6332
H-GCN-normH,0.813514,0.807843,0.740541,0.1958,0.3206
H-GCN,0.808108,0.843137,0.737838,0.6367,0.6752
W-GCN-normA,0.156757,0.037255,0.240541,0.2372,0.3284
W-GCN,0.689189,0.705882,0.710811,0.6246,0.7390


In [ ]:
table_acc_val1b

In [ ]:
table_acc1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.632432,0.605882,0.543243,0.6034,0.5634
Kipf-A,0.659459,0.564706,0.489189,0.6925,0.7963
A-GCN-normA,0.156757,0.037255,0.240541,0.7195,0.8117
A-GCN,0.894595,0.892157,0.840541,0.6078,0.6377
H-GCN-normH,0.878378,0.868627,0.802703,0.2370,0.3206
H-GCN,0.870270,0.886275,0.840541,0.6398,0.6789
W-GCN-normA,0.156757,0.037255,0.240541,0.2620,0.3297
W-GCN,0.789189,0.774510,0.781081,0.6324,0.7505


## Initial Params

In [ ]:
# BEST PARAMETERS
## Reaining params
N_RUNS = 20
N_EPOCHS = 200  # 500
LR = .01
WD = 5e-4
DROPOUT = .5

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3
HID_DIM = 16

## Model params
h0 = 1
ACT = nn.ReLU()
LAST_ACT = nn.LogSoftmax(dim=1)
LOSS_FN = nn.NLLLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': True, 'leg': 'A-GCN-normA'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': False, 'leg': 'A-GCN'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'H-GCN-normH'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'W-GCN'},]

In [ ]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)

            loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val2b = summary_table(accs_best_val2, DATASETS, EXPS)

TexasDataset
0: 0.676 (0.730) - 0.649 (0.649) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


0.108 (0.108) - 0.865 (0.892) - 0.622 (0.730) - 0.838 (0.892) - 0.108 (0.108) - 0.730 (0.757) - 
1: 0.595 (0.757) - 0.595 (0.595) - 0.189 (0.189) - 0.919 (0.973) - 0.919 (0.919) - 0.892 (0.946) - 0.189 (0.189) - 0.703 (0.784) - 
2: 0.486 (0.541) - 0.486 (0.486) - 0.216 (0.216) - 0.676 (0.784) - 0.595 (0.595) - 0.784 (0.892) - 0.216 (0.216) - 0.568 (0.595) - 
3: 0.676 (0.703) - 0.622 (0.622) - 0.108 (0.108) - 0.838 (0.892) - 0.676 (0.703) - 0.784 (0.919) - 0.108 (0.108) - 0.757 (0.784) - 
4: 0.568 (0.649) - 0.622 (0.622) - 0.189 (0.189) - 0.730 (0.865) - 0.649 (0.730) - 0.811 (0.865) - 0.189 (0.189) - 0.676 (0.676) - 
5: 0.622 (0.730) - 0.568 (0.568) - 0.189 (0.189) - 0.838 (0.919) - 0.703 (0.730) - 0.811 (0.892) - 0.189 (0.189) - 0.703 (0.703) - 
6: 0.622 (0.622) - 0.541 (0.622) - 0.081 (0.081) - 0.784 (0.865) - 0.568 (0.649) - 0.811 (0.919) - 0.081 (0.081) - 0.649 (0.676) - 
7: 0.676 (0.676) - 0.622 (0.622) - 0.108 (0.108) - 0.838 (0.838) - 0.649 (0.784) - 0.838 (0.865) - 0.108 (0.108

In [ ]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.616216,0.586275,0.478378,0.2087,0.3186
Kipf-A,0.605405,0.476471,0.372973,0.5830,0.7624
A-GCN-normA,0.156757,0.037255,0.240541,0.7193,0.8110
A-GCN,0.781081,0.823529,0.762162,0.5479,0.5537
H-GCN-normH,0.662162,0.688235,0.597297,0.2011,0.3189
GCN-H,0.808108,0.825490,0.745946,0.6058,0.5433
W-GCN-normA,0.156757,0.037255,0.240541,0.2215,0.3190
W-GCN,0.675676,0.617647,0.559459,0.5393,0.4424


In [ ]:
table_acc_val2b

In [ ]:
table_acc2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.681081,0.645098,0.564865,0.2392,0.3193
Kipf-A,0.613514,0.498039,0.418919,0.5866,0.7708
A-GCN-normA,0.156757,0.037255,0.240541,0.7281,0.8167
A-GCN,0.856757,0.896078,0.827027,0.5546,0.5569
H-GCN-normH,0.727027,0.756863,0.659459,0.2383,0.3195
GCN-H,0.883784,0.888235,0.845946,0.6110,0.5495
W-GCN-normA,0.156757,0.037255,0.240541,0.2472,0.3219
W-GCN,0.716216,0.690196,0.616216,0.5428,0.4448


## Best params - GF-GNN

In [ ]:
# BEST PARAMETERS
N_RUNS = 20
N_EPOCHS = 200  # 100
EPOCHS_h = 25
EPOCHS_W = 25
LR = .005
WD = .001
DROPOUT = .25

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3  # 2
HID_DIM = 32  # 32

## Model params
h0 = 1  # 1

IN_DIM = feat.shape[1]
OUT_DIM = n_class

ACT = nn.LeakyReLU() 
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': True, 'leg': 'A-GCN-normA'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': False, 'leg': 'A-GCN'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'H-GCN-normH'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'W-GCN'},]

In [ ]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD, epochs_h=EPOCHS_h,
                                        epochs_W=EPOCHS_W)
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val2b = summary_table(accs_best_val2, DATASETS, EXPS)

TexasDataset
0: 0.649 (0.649) - 0.108 (0.135) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


0.108 (0.108) - 0.784 (0.865) - 0.811 (0.838) - 0.784 (0.892) - 0.108 (0.108) - 0.703 (0.757) - 
1: 0.568 (0.649) - 0.162 (0.189) - 0.189 (0.189) - 0.919 (0.973) - 0.784 (0.919) - 0.946 (0.973) - 0.189 (0.189) - 0.703 (0.757) - 
2: 0.486 (0.486) - 0.568 (0.568) - 0.216 (0.216) - 0.811 (0.892) - 0.541 (0.622) - 0.811 (0.892) - 0.216 (0.216) - 0.622 (0.622) - 
3: 0.622 (0.622) - 0.676 (0.703) - 0.108 (0.108) - 0.757 (0.892) - 0.703 (0.730) - 0.865 (0.919) - 0.108 (0.108) - 0.703 (0.757) - 
4: 0.541 (0.622) - 0.622 (0.703) - 0.189 (0.189) - 0.811 (0.838) - 0.568 (0.703) - 0.892 (0.892) - 0.189 (0.189) - 0.649 (0.676) - 
5: 0.568 (0.568) - 0.568 (0.568) - 0.189 (0.189) - 0.838 (0.892) - 0.649 (0.784) - 0.784 (0.919) - 0.189 (0.189) - 0.595 (0.676) - 
6: 0.595 (0.595) - 0.568 (0.568) - 0.081 (0.081) - 0.838 (0.892) - 0.649 (0.757) - 0.757 (0.892) - 0.081 (0.081) - 0.649 (0.703) - 
7: 0.622 (0.676) - 0.108 (0.108) - 0.108 (0.108) - 0.811 (0.892) - 0.595 (0.622) - 0.703 (0.865) - 0.108 (0.108

In [ ]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.586486,0.517647,0.462162,0.2113,0.3019
Kipf-A,0.456757,0.503922,0.337838,0.5241,0.7144
A-GCN-normA,0.156757,0.037255,0.240541,0.7048,0.8065
A-GCN,0.829730,0.823529,0.751351,0.5750,0.5872
H-GCN-normH,0.664865,0.678431,0.627027,0.1811,0.3190
GCN-H,0.810811,0.847059,0.764865,0.6081,0.5169
W-GCN-normA,0.156757,0.037255,0.240541,0.2133,0.3188
W-GCN,0.659459,0.647059,0.559459,0.5106,0.4391


In [ ]:
table_acc2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.608108,0.547059,0.497297,0.2395,0.3026
Kipf-A,0.475676,0.509804,0.378378,0.5320,0.7196
A-GCN-normA,0.156757,0.037255,0.240541,0.7161,0.8114
A-GCN,0.891892,0.892157,0.832432,0.5794,0.5891
H-GCN-normH,0.735135,0.749020,0.670270,0.2312,0.3190
GCN-H,0.897297,0.888235,0.837838,0.6131,0.5176
W-GCN-normA,0.156757,0.037255,0.240541,0.2538,0.3205
W-GCN,0.705405,0.686275,0.594595,0.5231,0.4443


## Best params - GF-GNN norm H

In [ ]:
# BEST PARAMETERS
N_RUNS = 20
N_EPOCHS = 200  # 5000
EPOCHS_h = 25 # 5 # 5
EPOCHS_W = 25 # 5 # 25
LR = .005
WD = .001  # .001
DROPOUT = .25

# BEST PARAMETERS
## Architecture params
N_LAYERS = 3
K = 2
HID_DIM = 50 # 100

## Model params
NORM = True

IN_DIM = feat.shape[1]
OUT_DIM = n_class

ACT = nn.LeakyReLU()  # nn.ELU()
LAST_ACT = nn.Softmax()
LOSS_FN = nn.CrossEntropyLoss()  # nn.CrossEntropyLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': True, 'leg': 'A-GCN-normA'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': False, 'leg': 'A-GCN'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'H-GCN-normH'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'W-GCN'},]

In [ ]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD, epochs_h=EPOCHS_h,
                                        epochs_W=EPOCHS_W)
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val2b = summary_table(accs_best_val2, DATASETS, EXPS)

TexasDataset
0: 0.622 (0.703) - 0.730 (0.730) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


0.108 (0.108) - 0.757 (0.811) - 0.757 (0.865) - 0.757 (0.811) - 0.108 (0.108) - 0.811 (0.838) - 
1: 0.622 (0.649) - 0.676 (0.730) - 0.189 (0.189) - 0.811 (0.865) - 0.838 (0.973) - 0.757 (0.865) - 0.189 (0.189) - 0.784 (0.838) - 
2: 0.486 (0.541) - 0.568 (0.568) - 0.216 (0.216) - 0.676 (0.703) - 0.757 (0.865) - 0.676 (0.703) - 0.216 (0.216) - 0.676 (0.703) - 
3: 0.622 (0.676) - 0.649 (0.649) - 0.108 (0.108) - 0.865 (0.865) - 0.784 (0.919) - 0.865 (0.865) - 0.108 (0.108) - 0.676 (0.757) - 
4: 0.541 (0.595) - 0.595 (0.703) - 0.189 (0.189) - 0.784 (0.784) - 0.865 (0.919) - 0.784 (0.784) - 0.189 (0.189) - 0.757 (0.838) - 
5: 0.703 (0.703) - 0.703 (0.730) - 0.189 (0.189) - 0.811 (0.811) - 0.838 (0.892) - 0.784 (0.892) - 0.189 (0.189) - 0.757 (0.838) - 
6: 0.541 (0.595) - 0.568 (0.595) - 0.081 (0.081) - 0.622 (0.730) - 0.784 (0.865) - 0.622 (0.730) - 0.081 (0.081) - 0.703 (0.757) - 
7: 0.622 (0.649) - 0.676 (0.676) - 0.108 (0.108) - 0.865 (0.865) - 0.811 (0.865) - 0.811 (0.865) - 0.108 (0.108

In [ ]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.610811,0.601961,0.497297,0.2018,0.3188
Kipf-A,0.640541,0.545098,0.454054,0.6427,0.7597
A-GCN-normA,0.156757,0.037255,0.240541,0.7115,0.7853
A-GCN,0.775676,0.835294,0.732432,0.6599,0.7579
H-GCN-normH,0.791892,0.833333,0.745946,0.2072,0.2973
GCN-H,0.762162,0.833333,0.737838,0.6723,0.7575
W-GCN-normA,0.156757,0.037255,0.240541,0.2193,0.3185
W-GCN,0.740541,0.766667,0.724324,0.3260,0.7015


In [ ]:
table_acc2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.651351,0.631373,0.567568,0.2448,0.3190
Kipf-A,0.678378,0.596078,0.532432,0.6458,0.7684
A-GCN-normA,0.156757,0.037255,0.240541,0.7167,0.7881
A-GCN,0.805405,0.856863,0.767568,0.6748,0.7710
H-GCN-normH,0.889189,0.876471,0.824324,0.2412,0.2978
GCN-H,0.813514,0.858824,0.770270,0.6763,0.7694
W-GCN-normA,0.156757,0.037255,0.240541,0.2529,0.3195
W-GCN,0.794595,0.825490,0.764865,0.3337,0.7115


## Best params - GF-GNN no h

In [ ]:
# BEST PARAMETERS
N_RUNS = 20
N_EPOCHS = 500  # 500
LR = .05  # .01
WD = .005  # .005
DROPOUT = .25

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3  # 2
HID_DIM = 32 # 8

## Model params
h0 = 1  # 1
NORM = False

IN_DIM = feat.shape[1]
OUT_DIM = n_class

ACT = nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss()


EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': True, 'leg': 'A-GCN-normA'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': False, 'leg': 'A-GCN'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'H-GCN-normH'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'W-GCN'},]

In [ ]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD, epochs_h=EPOCHS_h,
                                        epochs_W=EPOCHS_W)
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val2b = summary_table(accs_best_val2, DATASETS, EXPS)

TexasDataset
0: 0.622 (0.703) - 0.730 (0.730) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:43: RuntimeWarning: divide by zero encountered in divide
  D_inv = np.diag(1/S.sum(1))
/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:51: RuntimeWarning: invalid value encountered in matmul
  return D_inv_sqr @ S @ D_inv_sqr


0.108 (0.108) - 0.811 (0.919) - 0.838 (0.865) - 0.811 (0.919) - 0.108 (0.108) - 0.730 (0.784) - 
1: 0.568 (0.703) - 0.595 (0.649) - 0.189 (0.189) - 0.946 (1.000) - 0.676 (0.838) - 0.946 (0.973) - 0.189 (0.189) - 0.784 (0.784) - 
2: 0.486 (0.541) - 0.514 (0.568) - 0.216 (0.216) - 0.811 (0.892) - 0.541 (0.622) - 0.811 (0.892) - 0.216 (0.216) - 0.595 (0.622) - 
3: 0.676 (0.703) - 0.649 (0.649) - 0.108 (0.108) - 0.838 (0.919) - 0.676 (0.703) - 0.757 (0.919) - 0.108 (0.108) - 0.703 (0.757) - 
4: 0.568 (0.649) - 0.622 (0.676) - 0.189 (0.189) - 0.838 (0.919) - 0.622 (0.649) - 0.838 (0.946) - 0.189 (0.189) - 0.622 (0.730) - 
5: 0.676 (0.730) - 0.703 (0.703) - 0.189 (0.189) - 0.784 (0.919) - 0.622 (0.757) - 0.838 (0.919) - 0.189 (0.189) - 0.622 (0.676) - 
6: 0.568 (0.622) - 0.568 (0.568) - 0.081 (0.081) - 0.811 (0.919) - 0.568 (0.595) - 0.703 (0.865) - 0.081 (0.081) - 0.649 (0.676) - 
7: 0.622 (0.676) - 0.649 (0.649) - 0.108 (0.108) - 0.838 (0.892) - 0.622 (0.730) - 0.865 (0.865) - 0.108 (0.108

In [ ]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.602703,0.623529,0.470270,0.2029,0.3168
Kipf-A,0.632432,0.539216,0.445946,0.5940,0.7755
A-GCN-normA,0.156757,0.037255,0.240541,0.6265,0.8061
A-GCN,0.829730,0.839216,0.759459,0.6517,0.6777
H-GCN-normH,0.648649,0.717647,0.678378,0.1981,0.3212
GCN-H,0.808108,0.835294,0.732432,0.6567,0.7064
W-GCN-normA,0.156757,0.037255,0.240541,0.2258,0.3192
W-GCN,0.681081,0.649020,0.594595,0.5968,0.6282


In [ ]:
table_acc2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.675676,0.668627,0.610811,0.2446,0.3194
Kipf-A,0.651351,0.570588,0.486486,0.6025,0.7859
A-GCN-normA,0.156757,0.037255,0.240541,0.6412,0.8215
A-GCN,0.916216,0.911765,0.851351,0.6582,0.6827
H-GCN-normH,0.724324,0.794118,0.770270,0.2385,0.3228
GCN-H,0.908108,0.911765,0.867568,0.6604,0.7108
W-GCN-normA,0.156757,0.037255,0.240541,0.2502,0.3199
W-GCN,0.718919,0.711765,0.648649,0.6046,0.6298
